# SISTER workflow

In [132]:
import json
import os
import xml.etree.ElementTree as ET
import pandas as pd
import IPython

# Import warnings module and ignore warnings in output below
import warnings
warnings.filterwarnings("ignore")

# Import and initialize MAAP class
from maap.maap import MAAP
maap = MAAP(maap_host="sister-api.imgspec.org")

### Create unique scene identifier

In [133]:
granules = ['https://avng.jpl.nasa.gov/avng/y16_data/ang20160101t060341.tar.gz']

meta = 'EXAMPLE_RUN'

scenes = []

crid = "990"

for l1_granule in granules:
    
    landsat = 'None'   

    base_name = os.path.basename(l1_granule)

    if base_name.startswith('DESIS'):
        sensor = 'DESIS'
        datetime = base_name[31:46]

    elif base_name.startswith('PRS'):
        sensor = 'PRISMA'
        datetime = base_name[16:24] + 'T' + base_name[24:30]
        landsat='https://sister-ops-workspace.s3.us-west-2.amazonaws.com/prisma/landsat_reference/PRS_%s_landsat.tar.gz' % base_name[16:50]

    elif base_name.startswith('ang'):
        sensor = 'AVNG'
        datetime = base_name[3:18]

    elif base_name.startswith('f'):
        sensor = 'AVCL'
        ''' AVIRIS classic filenames do not contain acquisition times,to be consistent with other
            sensors and to ensure identifier codes are unique a time string is created using other
            numbers in the filename            
        '''     

        datetime = "20%sT%s%s%s" % (base_name[1:7],
                                    base_name[8:10],
                                    base_name[11:13],
                                    base_name[14:16])
    else:
        raise ValueError('Unrecognized L1 datafile')

    job_args = {'sensor': sensor,
                'datetime': datetime,
                 'crid' : crid}
    job_args['preprocess'] = {'raw_dataset': l1_granule,
                              'landsat_dataset' : landsat}

    print(job_args)
    
    scenes.append(job_args)


{'sensor': 'AVNG', 'datetime': '20160101t060341', 'crid': '990', 'preprocess': {'raw_dataset': 'https://avng.jpl.nasa.gov/avng/y16_data/ang20160101t060341.tar.gz', 'landsat_dataset': 'None'}}


## Step 1. Preprocess

In [134]:
for i,scene in enumerate(scenes):
        
    if scene['sensor'] == 'AVCL':
        queue="sister-job_worker-32gb"
    else:
        queue="sister-job_worker-16gb"
    
    identifier = f'SISTER_{scene["sensor"]}_L1B_RDN_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    preprocess_job_response = maap.submitJob(
        algo_id = "sister-preprocess",
        version = "sister-dev",
        raw_dataset = scene['preprocess']['raw_dataset'],
        landsat_dataset = scene['preprocess']['landsat_dataset'],
        crid = scene['crid'],
        publish_to_cmr = False,
        cmr_metadata={},
        queue=queue,
        identifier= identifier)

    print(f'Identifier: {identifier}')
    print(f'Submission status: {preprocess_job_response.status}')
    print(f'Job ID: {preprocess_job_response.id}')
          
    scenes[i]['preprocess']['job_id'] = preprocess_job_response.id
    

Identifier: SISTER_AVNG_L1B_RDN_20160101t060341_990_EXAMPLE_RUN
Submission status: success
Job ID: 0ebcf868-5b9c-4a0f-a940-e66fae668b84


## Step 2. ISOFIT


In [127]:
for i,scene in enumerate(scenes):
        
    if scene['sensor'] == 'AVCL':
        segmentation_size = 100
    else:
        segmentation_size = 100
    
    identifier = f'SISTER_{scene["sensor"]}_L2A_RFL_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    preprocess_id = scene['preprocess']['job_id'] 
    
    preprocess_result= [ x for x in maap.getJobResult(preprocess_id).outputs if x.startswith("s3://s3.") and "RDN" in x]
    preprocess_result.sort()
    l1b_rdn,l1b_loc, l1b_obs = preprocess_result
    
    scene['preprocess']['radiance_dataset'] =l1b_rdn
    scene['preprocess']['location_dataset'] =l1b_loc
    scene['preprocess']['observation_dataset'] =l1b_obs

    isofit_job_response = maap.submitJob(
                                    algo_id="sister-isofit",
                                    version="sister-dev",
                                    radiance_dataset=l1b_rdn,
                                    location_dataset = l1b_loc,
                                    observation_dataset = l1b_obs,
                                    segmentation_size = segmentation_size,
                                    n_cores=32,
                                    crid = scene['crid'],
                                    publish_to_cmr=False,
                                    cmr_metadata={},
                                    queue="sister-job_worker-32gb",
                                    identifier= identifier)

    print(f'Identifier: {identifier}')
    print(f'Submission status: {isofit_job_response.status}')
    print(f'Job ID: {isofit_job_response.id}')
    
    scene['isofit']  = {'job_id' : isofit_job_response.id}


Identifier: SISTER_DESIS_L2A_RFL_20200818T141910_990_VEGTESTING
Submission status: success
Job ID: 244c8081-d78a-4515-9d34-37a6eefd5c93
Identifier: SISTER_PRISMA_L2A_RFL_20201206T085950_990_VEGTESTING
Submission status: success
Job ID: 6b2b10d1-1c14-400c-b2bf-2b4e1c0fa642
Identifier: SISTER_AVCL_L2A_RFL_20080709T010008_990_VEGTESTING
Submission status: success
Job ID: 12efbaee-f84b-4af9-8835-7ceb01582931
Identifier: SISTER_AVNG_L2A_RFL_20160101t060341_990_VEGTESTING
Submission status: success
Job ID: 2d411eba-4819-4533-81d1-7bb9eea64b11


## Step 3. Spectral resample

In [128]:
for i,scene in enumerate(scenes):

    identifier = f'SISTER_{scene["sensor"]}_L2A_RSRFL_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    isofit_id = scene['isofit']['job_id'] 

    iso_result= [ x for x in maap.getJobResult(isofit_id).outputs if x.startswith("s3://s3.") and "RFL" in x]
    l2a_rfl,l2a_unc = iso_result
        
    scene['isofit']['reflectance_dataset'] =l2a_rfl
    scene['isofit']['uncertainty_dataset'] =l2a_unc

    resample_job_response = maap.submitJob(
                                            algo_id="sister-resample",
                                            version="sister-dev",
                                            reflectance_dataset= l2a_rfl,
                                            uncertainty_dataset= l2a_unc,
                                            crid = scene['crid'],
                                            publish_to_cmr=False,
                                            cmr_metadata={},
                                            queue="sister-job_worker-16gb",
                                            identifier=identifier)

    print(f'Identifier: {identifier}')
    print('Submission status: %s' % resample_job_response.status)
    print('Job ID: %s' % resample_job_response.id)
    scene['resample']  = {'job_id' : resample_job_response.id}


Identifier: SISTER_DESIS_L2A_RSRFL_20200818T141910_990_VEGTESTING
Submission status: success
Job ID: 64dc5d7a-1a5d-4b46-9c49-26998d4170a5
Identifier: SISTER_PRISMA_L2A_RSRFL_20201206T085950_990_VEGTESTING
Submission status: success
Job ID: a5e63e9e-48ca-4540-94c7-07e5e0e132f0
Identifier: SISTER_AVCL_L2A_RSRFL_20080709T010008_990_VEGTESTING
Submission status: success
Job ID: 5af424eb-6420-42b0-98d8-097947a248f0
Identifier: SISTER_AVNG_L2A_RSRFL_20160101t060341_990_VEGTESTING
Submission status: success
Job ID: 7274ff9b-04b8-4d15-be9f-ab0891fe9277


## Step 3. Reflectance correction

In [129]:
for i,scene in enumerate(scenes):

    identifier = f'SISTER_{scene["sensor"]}_L2A_CORFL_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    resample_id = scene['resample']['job_id'] 
    resample_result= [ x for x in maap.getJobResult(resample_id).outputs if x.startswith("s3://s3.") and "RSRFL" in x]
    l2a_rsrfl,l2a_rsunc = resample_result
    
    scene['resample']['reflectance_dataset'] =l2a_rsrfl
    scene['resample']['uncertainty_dataset'] =l2a_rsunc



    rfl_corr_job_response = maap.submitJob(
                                            algo_id="sister-reflect_correct",
                                            version="sister-dev",
                                            observation_dataset= scene['preprocess']['observation_dataset'],
                                            reflectance_dataset= l2a_rsrfl,
                                            crid = scene['crid'],
                                            publish_to_cmr=False,
                                            cmr_metadata={},
                                            queue="sister-job_worker-16gb",
                                            identifier=identifier)

    print(f'Identifier: {identifier}')
    print('Submission status: %s' % rfl_corr_job_response.status)
    print('Job ID: %s' % rfl_corr_job_response.id)
    scene['reflect_correct']  = {'job_id' : rfl_corr_job_response.id}


Identifier: SISTER_DESIS_L2A_CORFL_20200818T141910_990_VEGTESTING
Submission status: success
Job ID: bd3cae41-38cb-4cc0-bd98-60b68c3fb095
Identifier: SISTER_PRISMA_L2A_CORFL_20201206T085950_990_VEGTESTING
Submission status: success
Job ID: 7b44af37-3f74-455d-9d9a-063ed2b7542e
Identifier: SISTER_AVCL_L2A_CORFL_20080709T010008_990_VEGTESTING
Submission status: success
Job ID: c9aac477-982b-43fb-b28d-2414e4f987bf
Identifier: SISTER_AVNG_L2A_CORFL_20160101t060341_990_VEGTESTING
Submission status: success
Job ID: adad7605-b62c-4dd0-9c80-b28341a24129


## Step 4. Fractional Cover

In [130]:
for i,scene in enumerate(scenes):

    identifier = f'SISTER_{scene["sensor"]}_L2B_FRCOV_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    correct_id = scene['reflect_correct']['job_id'] 
    correct_result= [ x for x in maap.getJobResult(correct_id).outputs if x.startswith("s3://s3.") and "CORFL" in x]
    l2a_corfl =  correct_result[0]
    scene['reflect_correct']['reflectance_dataset'] = l2a_corfl

    frcover_job_response = maap.submitJob(
                                        algo_id="sister-fractional-cover",
                                        version="sister-dev",
                                        reflectance_dataset=l2a_corfl,
                                        n_cores= 20,
                                        refl_scale= 1,
                                        normalization = 'brightness',
                                        crid = scene['crid'],
                                        publish_to_cmr=False,
                                        cmr_metadata={},
                                        queue="sister-job_worker-32gb",
                                        identifier= identifier)

    print(f'Identifier: {identifier}')
    print('Submission status: %s' % frcover_job_response.status)
    print('Job ID: %s' % frcover_job_response.id)
    scene['frcover']  = {'job_id' : frcover_job_response.id}

Identifier: SISTER_DESIS_L2B_FRCOV_20200818T141910_990_VEGTESTING
Submission status: success
Job ID: d694537d-c958-4779-b726-329f76f6dfca
Identifier: SISTER_PRISMA_L2B_FRCOV_20201206T085950_990_VEGTESTING
Submission status: success
Job ID: eb9336c8-03e0-45ab-aec1-f9edd6bbb68c
Identifier: SISTER_AVCL_L2B_FRCOV_20080709T010008_990_VEGTESTING
Submission status: success
Job ID: bee8d402-d74c-45a2-ab8c-a29b340bd898
Identifier: SISTER_AVNG_L2B_FRCOV_20160101t060341_990_VEGTESTING
Submission status: success
Job ID: aaa318c7-f578-484c-8d13-88cfe0462296


## Step 6a. Vegetation biochemistry

In [131]:
for i,scene in enumerate(scenes):

    identifier = f'SISTER_{scene["sensor"]}_L2B_VEGBIOCHEM_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    frcover_id = scene['frcover']['job_id'] 
    frcover_result= [x for x in maap.getJobResult(frcover_id).outputs if x.startswith("s3://s3.") and "FRCOV" in x]
    l2b_frcov =  frcover_result[0]                   
    scene['frcover']['frcover_dataset'] = l2b_frcov

    vegbiochem_job_response = maap.submitJob(
                            algo_id="sister-trait_estimate",
                            version="sister-dev",
                            reflectance_dataset= scene['reflect_correct']['reflectance_dataset'],
                            frcov_dataset=l2b_frcov,
                            veg_cover = 0.5,
                            crid = scene['crid'],
                            publish_to_cmr=False,
                            cmr_metadata={},
                            queue="sister-job_worker-16gb",
                            identifier=identifier)

    print(f'Identifier: {identifier}')
    print('Submission status: %s' % vegbiochem_job_response.status)
    print('Job ID: %s' % vegbiochem_job_response.id)
    scene['vegbiochem']  = {'job_id' : vegbiochem_job_response.id}

Identifier: SISTER_DESIS_L2B_VEGBIOCHEM_20200818T141910_990_VEGTESTING
Submission status: success
Job ID: 782a103f-27bc-441f-86a9-c65313f604c3
Identifier: SISTER_PRISMA_L2B_VEGBIOCHEM_20201206T085950_990_VEGTESTING
Submission status: success
Job ID: 6cc1189a-437b-4280-a283-a18620671891
Identifier: SISTER_AVCL_L2B_VEGBIOCHEM_20080709T010008_990_VEGTESTING
Submission status: success
Job ID: 27148f66-5233-46d9-94f3-47f560bd7fa5
Identifier: SISTER_AVNG_L2B_VEGBIOCHEM_20160101t060341_990_VEGTESTING
Submission status: success
Job ID: 109b2435-9aca-4001-b8a1-41fd7f3ab0ec


## Step 6b. Snow grainsize

In [126]:
for i,scene in enumerate(scenes):

    identifier = f'SISTER_{scene["sensor"]}_L2B_GRAINSIZE_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    frcover_id = scene['frcover']['job_id'] 
    frcover_result= [x for x in maap.getJobResult(frcover_id).outputs if x.startswith("s3://s3.") and "FRCOV" in x]
    l2b_frcov =  frcover_result[0]                   
    scene['frcover']['frcover_dataset'] = l2b_frcov

    grainsize_job_response = maap.submitJob(
        algo_id="sister-grainsize",
        version="sister-dev",
        reflectance_dataset=  scene['reflect_correct']['reflectance_dataset'],
        frcov_dataset= scene['frcover']['frcover_dataset'],
        snow_cover = 0.9,
        crid= scene['crid'],
        publish_to_cmr=False,
        cmr_metadata={},
        queue="sister-job_worker-16gb",
        identifier=identifier)
    
    print(f'Identifier: {identifier}')
    print('Submission status: %s' % grainsize_job_response.status)
    print('Job ID: %s' % grainsize_job_response.id)
    scene['grainsize']  = {'job_id' : grainsize_job_response.id}

Identifier: SISTER_AVCL_L2B_GRAINSIZE_20110513T010003_099_VEGTESTING
Submission status: success
Job ID: 9e058f30-020b-4aa2-9da6-0fceb1db0f40


In [ ]:
scenes